![](https://img-d02.moneycontrol.co.in/images/common/header/logo.png?v=0.1?raw=true)

## Moneycontrol scraping
#### First we need to scrape all the companies under Indian Market

In [1]:
from multiprocessing.dummy import Pool
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### The function which scraps links like:
    - https://www.moneycontrol.com/india/stockpricequote/A
    - https://www.moneycontrol.com/india/stockpricequote/B
    - so on...
    - https://www.moneycontrol.com/india/stockpricequote/Z
    - https://www.moneycontrol.com/india/stockpricequote/others

In [2]:
company_list=[]
def get_company_list(url):    
    page_response = requests.get(url, timeout=5)
    soup = BeautifulSoup(page_response.content,"lxml")
    companies = soup.findAll("a", attrs={"class":"bl_12"})
    for company in companies:
        if "moneycontrol" in company["href"]:
            company_list.append([company.text,company["href"]])

#### This generates all links and sends to the function through multithreading

In [3]:
urls=[]
incomplete_url="https://www.moneycontrol.com/india/stockpricequote/"
for i in range (ord("A"),ord("Z")+1):
    urls.append(incomplete_url+chr(i))
urls.append((incomplete_url+"others"))
with Pool() as pool:
    (pool.map(get_company_list, urls))
df =  pd.DataFrame(company_list)
df.to_csv('company_list.csv', sep=',', header=None, index=None)